## Descargar el siguiente dataset.
**'titanic3.xls'**

Hacerle tratamiento a los valores faltantes de la columna Edad. Puede utilizar diversas técnicas al mismo tiempo. Indique en comentarios en el notebook el razonamiento que siguió para imputar los nulos.

In [1]:
# %pip install xlrd
import pandas as pd

df = pd.read_excel('../data/titanic3.xls')
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [2]:
df['age'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1309 entries, 0 to 1308
Series name: age
Non-Null Count  Dtype  
--------------  -----  
1046 non-null   float64
dtypes: float64(1)
memory usage: 10.4 KB


In [3]:
print("Valores faltantes en 'age':", df['age'].isna().sum())

Valores faltantes en 'age': 263


## Análisis para Imputación

Antes de imputar los valores faltantes, relizaremos un análisis las relaciones entre la edad y otras variables para establecer criterios de imputación.

In [4]:
# Analizar la distribución de edad por diferentes variables
print("\nEdad promedio por clase (pclass):")
print(df.groupby('pclass')['age'].agg(['mean', 'median', 'count']))

print("\nEdad promedio por sexo:")
print(df.groupby('sex')['age'].agg(['mean', 'median', 'count']))

print("\nEdad promedio por clase y sexo:")
print(df.groupby(['pclass', 'sex'])['age'].agg(['mean', 'median', 'count']))


Edad promedio por clase (pclass):
             mean  median  count
pclass                          
1       39.159918    39.0    284
2       29.506705    29.0    261
3       24.816367    24.0    501

Edad promedio por sexo:
             mean  median  count
sex                             
female  28.687071    27.0    388
male    30.585233    28.0    658

Edad promedio por clase y sexo:
                    mean  median  count
pclass sex                             
1      female  37.037594    36.0    133
       male    41.029250    42.0    151
2      female  27.499191    28.0    103
       male    30.815401    29.5    158
3      female  22.185307    22.0    152
       male    25.962273    25.0    349


In [5]:
# Analizar el título en el nombre
df['title'] = df['name'].apply(lambda name: name.split(',')[1].split('.')[0].strip()) # Extraer parámetros con split, luego eliminar espacios innecesarios

print("Títulos únicos encontrados:")
df['title'].value_counts()

Títulos únicos encontrados:


title
Mr              757
Miss            260
Mrs             197
Master           61
Dr                8
Rev               8
Col               4
Major             2
Mlle              2
Ms                2
Lady              1
Capt              1
Mme               1
Sir               1
Jonkheer          1
Dona              1
Don               1
the Countess      1
Name: count, dtype: int64

In [6]:
print("Edad promedio por título:")
print(df.groupby('title')['age'].agg(['mean', 'median', 'count']))

Edad promedio por título:
                   mean  median  count
title                                 
Capt          70.000000    70.0      1
Col           54.000000    54.5      4
Don           40.000000    40.0      1
Dona          39.000000    39.0      1
Dr            43.571429    49.0      7
Jonkheer      38.000000    38.0      1
Lady          48.000000    48.0      1
Major         48.500000    48.5      2
Master         5.482704     4.0     53
Miss          21.774207    22.0    210
Mlle          24.000000    24.0      2
Mme           24.000000    24.0      1
Mr            32.252151    29.0    581
Mrs           36.994118    35.5    170
Ms            28.000000    28.0      1
Rev           41.250000    41.5      8
Sir           49.000000    49.0      1
the Countess  33.000000    33.0      1


In [7]:
# Analizar la relación entre sibsp (hermanos/esposos) y parch (padres/hijos) con la edad
print("Edad promedio por número de hermanos/esposos (sibsp):")
print(df.groupby('sibsp')['age'].agg(['mean', 'median', 'count']))

print("\nEdad promedio por número de padres/hijos (parch):")
print(df.groupby('parch')['age'].agg(['mean', 'median', 'count']))

# Crear una variable que indique el tamaño de la familia
df['family_size'] = df['sibsp'] + df['parch']
# Crear una variable que indique si viaja solo o acompañado
df['is_alone'] = (df['family_size'] == 0)

Edad promedio por número de hermanos/esposos (sibsp):
            mean  median  count
sibsp                          
0      30.921776    28.0    685
1      31.058036    30.0    280
2      23.569444    21.5     36
3      16.312500    14.5     16
4       8.772727     7.0     22
5      10.166667    10.5      6
8      14.500000    14.5      1

Edad promedio por número de padres/hijos (parch):
            mean  median  count
parch                          
0      31.969401    29.0    768
1      24.965625    22.5    160
2      18.975945    17.0     97
3      38.875000    42.0      8
4      47.600000    45.0      5
5      39.333333    39.0      6
6      41.500000    41.5      2
9            NaN     NaN      0


In [8]:
# Verificar nuevas variables
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,title,family_size,is_alone
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",Miss,0,True
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",Master,3,False
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",Miss,3,False
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON",Mr,3,False
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",Mrs,3,False


In [9]:
print("Edad promedio según tamaño de familia:")
print(df.groupby('family_size')['age'].agg(['mean', 'median', 'count']))    

Edad promedio según tamaño de familia:
                  mean  median  count
family_size                          
0            31.511864    29.0    590
1            32.726942    30.0    206
2            26.534143    26.0    144
3            19.423079    24.0     39
4            23.764706    18.0     17
5            20.120000    14.0     25
6            17.375000    10.0     16
7            18.000000    12.5      8
10           14.500000    14.5      1


## Imputación de Valores Faltantes

Agregando Columnas con diferentes estrategias de imputación para la edad.

In [10]:
# Imputar con la media agrupada por SEXO
# Hombres y mujeres pueden tener edades promedio diferentes.
promedios_sexo = df.groupby("sex")["age"].mean().reset_index()
promedios_sexo.columns = ["sex", "age_mean_sex"]

print("Promedios por sexo:")
print(promedios_sexo)

Promedios por sexo:
      sex  age_mean_sex
0  female     28.687071
1    male     30.585233


In [11]:
# Hacer merge con el dataframe original
df = df.merge(promedios_sexo, on=["sex"], how="left")

# Imputar los valores nulos con el promedio del grupo
df["age(imp:mean_sex)"] = df["age"].fillna(df["age_mean_sex"])

print("Media agrupada por sexo:")
print(f"Valores faltantes restantes: {df['age(imp:mean_sex)'].isna().sum()}")

Media agrupada por sexo:
Valores faltantes restantes: 0


In [12]:
# Imputar con la media agrupada por clase (pclass)
# La clase socioeconómica está relacionada con la edad
promedios_clase = df.groupby("pclass")["age"].mean().reset_index()
promedios_clase.columns = ["pclass", "age_mean_pclass"]

print("Promedios por clase:")
print(promedios_clase)

Promedios por clase:
   pclass  age_mean_pclass
0       1        39.159918
1       2        29.506705
2       3        24.816367


In [13]:
# Hacer merge con el dataframe original
df = df.merge(promedios_clase, on=["pclass"], how="left")

# Imputar los valores nulos con el promedio del grupo
df["age(imp:mean_pclass)"] = df["age"].fillna(df["age_mean_pclass"])

print("Media agrupada por clase:")
print(f"Valores faltantes restantes: {df['age(imp:mean_pclass)'].isna().sum()}")

Media agrupada por clase:
Valores faltantes restantes: 0


In [14]:
# Imputar con la media agrupada por título
# Los títulos (Mr, Mrs, Miss, Master) se relacionan con la edad
promedios_titulo = df.groupby("title")["age"].mean().reset_index()
promedios_titulo.columns = ["title", "age_mean_title"]

print("Promedios por título:")
print(promedios_titulo)

Promedios por título:
           title  age_mean_title
0           Capt       70.000000
1            Col       54.000000
2            Don       40.000000
3           Dona       39.000000
4             Dr       43.571429
5       Jonkheer       38.000000
6           Lady       48.000000
7          Major       48.500000
8         Master        5.482704
9           Miss       21.774207
10          Mlle       24.000000
11           Mme       24.000000
12            Mr       32.252151
13           Mrs       36.994118
14            Ms       28.000000
15           Rev       41.250000
16           Sir       49.000000
17  the Countess       33.000000


In [15]:
# Hacer merge con el dataframe original
df = df.merge(promedios_titulo, on=["title"], how="left")

# Imputar los valores nulos con el promedio del grupo
df["age(imp:mean_title)"] = df["age"].fillna(df["age_mean_title"])

print("Media agrupada por título:")
print(f"Valores faltantes restantes: {df['age(imp:mean_title)'].isna().sum()}")

Media agrupada por título:
Valores faltantes restantes: 0


In [16]:
# Imputar con la media agrupada por sexo + clase
# Combinación de sexo y clase para mejor precisión
promedios_sexo_clase = df.groupby(["sex", "pclass"])["age"].mean().reset_index()
promedios_sexo_clase.columns = ["sex", "pclass", "age_mean_sex_pclass"]

print("Promedios por sexo y clase:")
print(promedios_sexo_clase)

Promedios por sexo y clase:
      sex  pclass  age_mean_sex_pclass
0  female       1            37.037594
1  female       2            27.499191
2  female       3            22.185307
3    male       1            41.029250
4    male       2            30.815401
5    male       3            25.962273


In [17]:
# Hacer merge con el dataframe original
df = df.merge(promedios_sexo_clase, on=["sex", "pclass"], how="left")

# Imputar los valores nulos con el promedio del grupo
df["age(imp:mean_sex_pclass)"] = df["age"].fillna(df["age_mean_sex_pclass"])

print("Media agrupada por sexo y clase:")
print(f"Valores faltantes restantes: {df['age(imp:mean_sex_pclass)'].isna().sum()}")

Media agrupada por sexo y clase:
Valores faltantes restantes: 0


In [24]:
# Ver las columnas finales del dataframe
print("Columnas finales del dataframe:", df.columns)

Columnas finales del dataframe: Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest', 'title',
       'family_size', 'is_alone', 'age_mean_sex', 'age(imp:mean_sex)',
       'age_mean_pclass', 'age(imp:mean_pclass)', 'age_mean_title',
       'age(imp:mean_title)', 'age_mean_sex_pclass',
       'age(imp:mean_sex_pclass)'],
      dtype='object')


In [19]:
# Ver el dataframe final con todas las columnas
df.head(20)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,...,family_size,is_alone,age_mean_sex,age(imp:mean_sex),age_mean_pclass,age(imp:mean_pclass),age_mean_title,age(imp:mean_title),age_mean_sex_pclass,age(imp:mean_sex_pclass)
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,...,0,True,28.687071,29.000000,39.159918,29.000000,21.774207,29.000000,37.037594,29.00000
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,...,3,False,30.585233,0.916700,39.159918,0.916700,5.482704,0.916700,41.029250,0.91670
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,...,3,False,28.687071,2.000000,39.159918,2.000000,21.774207,2.000000,37.037594,2.00000
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,...,3,False,30.585233,30.000000,39.159918,30.000000,32.252151,30.000000,41.029250,30.00000
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,...,3,False,28.687071,25.000000,39.159918,25.000000,36.994118,25.000000,37.037594,25.00000
5,1,1,"Anderson, Mr. Harry",male,48.0000,0,0,19952,26.5500,E12,...,0,True,30.585233,48.000000,39.159918,48.000000,32.252151,48.000000,41.029250,48.00000
6,1,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1,0,13502,77.9583,D7,...,1,False,28.687071,63.000000,39.159918,63.000000,21.774207,63.000000,37.037594,63.00000
7,1,0,"Andrews, Mr. Thomas Jr",male,39.0000,0,0,112050,0.0000,A36,...,0,True,30.585233,39.000000,39.159918,39.000000,32.252151,39.000000,41.029250,39.00000
8,1,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2,0,11769,51.4792,C101,...,2,False,28.687071,53.000000,39.159918,53.000000,36.994118,53.000000,37.037594,53.00000
9,1,0,"Artagaveytia, Mr. Ramon",male,71.0000,0,0,PC 17609,49.5042,NaN,...,0,True,30.585233,71.000000,39.159918,71.000000,32.252151,71.000000,41.029250,71.00000
